In [1]:
import re
import tqdm
from pprint import pprint
import requests
import argparse
from bs4 import BeautifulSoup
from urllib.parse import quote

keyword = '遠傳'
max_len = 256

In [2]:
# define the headers of browsers
headers = {
    "User-Agent":'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:97.0) Gecko/20100101 Firefox/97.0'
}

# search your keywords with target url
url = 'https://tw.finance.yahoo.com/news_search.html?ei=Big5&q=' + quote(keyword.encode('big5'))
res = requests.get(url, headers=headers)
soup = BeautifulSoup(res.content, "html.parser")

# get the link of each news and turn into soup object
tag_name = 'span a'
url_news = [link['href'] for link in soup.select(tag_name) if link['href'] != '#']
soup_news = [BeautifulSoup(requests.get(link, headers=headers).content, "html.parser") 
             for link in tqdm.tqdm(url_news)]

# retrieve and clean the content of news
text_news = [[re.sub(r'(\([^()]*\))|(【[^【】]*】)|(（[^（）]*）)|(\s)', '', text.text) 
              for text in news.find_all('p') if 'class' not in str(text)] 
             for news in soup_news
             if '【公告】' not in str(news.select('header h1'))]

# define a recursive function to split and concatenate each news based on the maximum length of our model

def combine_text(texts, article):
    text_concat = ''
    # text: each paragraph of this news
    for text in texts:
        # for those paragraphs already greater than max_len, recursively use this function  
        if len(text) > max_len:
            article += combine_text(text.split('，'), article)
        # try to fill text_concat to length max_len 
        elif len(text_concat+text) <= max_len:
            text_concat += text
        # append and reset text_concat if it reaches max_len
        else:
            article.append(text_concat)
            text_concat = ''
    return article

articles = []
for texts in text_news:
    article = combine_text(texts, [])
    if not len(article): continue
    articles.append(article)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.69it/s]


In [3]:
articles

[['今日上午台電無預警發生跳電，導致全台多處地區大停電。目前已知不少地區交通燈號停擺，高雄捷運也出現停止營運的狀況。在各地賣場方面，超市龍頭全聯福利中心表示，部分門市有受到影響，但各門市原本就配有發電機，狀況還在控制範圍內。而愛買表示目前正在營運中，家樂福表示，各店均正常營運，平時亦依照程序落實機電設備保養，確保消費者及員工安全。',
  '而百貨通路方面，SOGO回應，高雄店已恢復電力，預計12：30開店，其他全台分店都正常營業。而新光三越南部分店與SKMPark延後開店，其中高雄三多店已於中午12時復電，高雄左營店稍早也復電，並於下午2時15分開店營業，台南新天地也於下午3時20分開店，台南中山店在下午3點、SKMPark於下午4點半開店營業，目前新光三越旗下全台各店皆已復電恢復營業。。環球百貨表示，GlobalMall桃園A8、林口A9、屏東市及新左營車站四店都已全部恢復電力且營業。'],
 ['近期電信市場接連發生購併案先是台灣大在去年底宣布合併台灣之星上週則是遠傳合併亞太電信中華信評認為上述兩宗合併案獲得主管機關核准4G吃到飽方案相關的價格競爭將獲得顯著緩解較佳的市場紀律下5G數據方案的價格競爭會比4G時代更為緩和。由於台灣之星、亞太電信進入市場的時間較晚兩家公司一直透過更具競爭力的資費價格來取得市占率導致市場上其他既有電信業者的資費價格在過去幾年顯著下滑中華信評認為國內三大電信業者在市場整合後將側重於透過服務升級以維持較佳的EBITDA利潤率而非藉由價格競爭來取得市占率。遠傳併了亞太電信',
  '近期電信市場接連發生購併案先是台灣大在去年底宣布合併台灣之星上週則是遠傳合併亞太電信中華信評認為上述兩宗合併案獲得主管機關核准4G吃到飽方案相關的價格競爭將獲得顯著緩解較佳的市場紀律下5G數據方案的價格競爭會比4G時代更為緩和。由於台灣之星、亞太電信進入市場的時間較晚兩家公司一直透過更具競爭力的資費價格來取得市占率導致市場上其他既有電信業者的資費價格在過去幾年顯著下滑中華信評認為國內三大電信業者在市場整合後將側重於透過服務升級以維持較佳的EBITDA利潤率而非藉由價格競爭來取得市占率。遠傳併了亞太電信'],
 ['全台今日上午傳出大規模停電，傳出部分地區的網路和通話受到影響，對此，五大電信商回應，全台基地台均有備援電池，北部地區已陸續供電，暫無影響，門市與通